In [1]:
!pip install yt-dlp
!pip install langchain
!pip install openai

### Create function for retrieving the duration of a YouTube playlist

In [2]:
import yt_dlp

In [40]:
def seconds_to_hms(seconds: int) -> str:
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f'{hours:02d}:{minutes:02d}:{seconds:02d}'

def get_total_playlist_duration(playlist_url: str) -> str:
    ydl = yt_dlp.YoutubeDL(params={'quiet': True})
    info = ydl.extract_info(url=playlist_url, download=False)
    video_durations = [video['duration'] for video in info['entries']]
    return seconds_to_hms(sum(video_durations))

### Use LangChain for creating an agent that interacts with a custom tool

In [59]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, tools
from langchain.agents import AgentType, Tool

In [60]:
OPENAI_API_KEY = 'YOUR_API_KEY'

In [57]:
@tool
def ytpd(playlist_url: str) -> str:
    """Returns the duration of YouTube playlist with URL {playlist_url}. \
    Use this tool for any questions related to finding the duration of a \
    YouTube playlist. The input is {playlist_url} the URL to the YouTube \
    playlist, and the output is the playlist duration as a string in the \
    `HH:MM:SS` format."""
    return get_total_playlist_duration(playlist_url)

tools = [Tool(name = "YTPD",
              func=ytpd,
              description="Returns the duration of YouTube playlist with URL {playlist_url}. \
                    Use this tool for any questions related to finding the duration of a \
                    YouTube playlist. The input is {playlist_url} the URL to the YouTube \
                    playlist, and the output is the playlist duration as a string in the \
                    `HH:MM:SS` format."
              )]

In [61]:
llm = ChatOpenAI(temperature=0,
                 openai_api_key=OPENAI_API_KEY)

In [68]:
agent = initialize_agent(tools, llm,
                         agent=AgentType.OPENAI_FUNCTIONS,
                         verbose=True)

### Interact with agent

In [74]:
playlist_urls = ['https://www.youtube.com/playlist?list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab',
                 'https://www.youtube.com/playlist?list=PLZHQObOWTQDMsr9K-rj53DwVRMYO3t5Yr',
                 'https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi'
                 ]

In [75]:
agent.run(f'What is the duration of this playlist?: {playlist_urls[-1]}')



> Entering new AgentExecutor chain...

Invoking: `YTPD` with `{'playlist_url': 'https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi'}`


01:02:16The duration of the playlist is 01:02:16.

> Finished chain.


'The duration of the playlist is 01:02:16.'

In [76]:
agent.run(f'What is the duration of these two playlists?: {playlist_urls[0]} and {playlist_urls[1]}')



> Entering new AgentExecutor chain...

Invoking: `YTPD` with `{'playlist_url': 'https://www.youtube.com/playlist?list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab'}`


03:00:45
Invoking: `YTPD` with `{'playlist_url': 'https://www.youtube.com/playlist?list=PLZHQObOWTQDMsr9K-rj53DwVRMYO3t5Yr'}`


03:11:01The duration of the first playlist is 03:00:45 and the duration of the second playlist is 03:11:01.

> Finished chain.


'The duration of the first playlist is 03:00:45 and the duration of the second playlist is 03:11:01.'

In [78]:
agent.run(f'What is the duration of all three playlists combined? Here are the links to the playlists: {playlist_urls[0]} and {playlist_urls[1]} and {playlist_urls[2]}')



> Entering new AgentExecutor chain...

Invoking: `YTPD` with `{'playlist_url': 'https://www.youtube.com/playlist?list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab'}`


03:00:45
Invoking: `YTPD` with `{'playlist_url': 'https://www.youtube.com/playlist?list=PLZHQObOWTQDMsr9K-rj53DwVRMYO3t5Yr'}`


03:11:01
Invoking: `YTPD` with `{'playlist_url': 'https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi'}`


01:02:16The duration of all three playlists combined is 07:14:02.

> Finished chain.


'The duration of all three playlists combined is 07:14:02.'